In [172]:
import pandas


In [173]:
doc = []
with open ('dates.txt') as file:
    for line in file:
        doc.append(line)
data = pandas.Series(doc)
data.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [174]:
data.describe

<bound method NDFrame.describe of 0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object>

In [175]:
#cardinality
ZERO_OR_MORE = '*'
ONE_OR_MORE = '+'
ZERO_OR_ONE = '?'
EXACTLY_TWO = "{2}"
ONE_OR_TWO = "{1,2}"
EXACTLY_ONE = '{1}'

In [176]:
#GROUPS
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = "[{}]"
NEGATIVE_LOOKAHEAD = "(?!{})"
NEGATIVE_LOOKBEHIND = "(?<!{})"
POSITIVE_LOOKAHEAD = "(?={})"
POSITIVE_LOOKBEHIND = "(?<={})"
ESCAPE = "\{}"

In [177]:
#NUMBERS
DIGIT = r"\d"
ONE_DIGIT = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + "{3}"
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + R"{4}" + NON_DIGIT

In [178]:
#STRING LITERALS
SLASH = r"/"
OR = r'|'
LOWER_CASE = "a-z"
SPACE = "\s"
DOT = "."
DASH = "-"
COMMA = ","
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
EMPTY_STRING = ""


In [179]:
#DATES
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS


In [180]:
#example jan 1st, 2022
CONTRACTED = (ONE_OR_TWO_DIGITS 
             + LOWER_CASE 
             + EXACTLY_TWO)
CONTRACTION = NAMED.format("contraction",
                          MONTHS 
                          + CONTRACTED 
                          + DAY_SUFFIX
                          + YEAR)

In [181]:
#example Jan 2022
NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)

In [182]:
#Example Jan 1st, 2022
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)

In [183]:
BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)

In [184]:
slashed = SLASH.join ([ONE_OR_TWO_DIGITS, 
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
dashed = DASH.join([ONE_OR_TWO_DIGITS,
                   ONE_OR_TWO_DIGITS,
                   EXACTLY_TWO_DIGITS])
TWENTIETH_CENTURY = NAMED.format("twentieth",
                                 OR.join([slashed, dashed]))

In [185]:
NUMERIC = NAMED.format("numeric",
                      SLASH.join([ONE_OR_TWO_DIGITS,
                                 ONE_OR_TWO_DIGITS,
                                 FOUR_DIGITS]))

In [186]:
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
             + ONE_DIGIT
             + SLASH
             + FOUR_DIGITS)
NO_TWO_DAYS = (NO_PRECEDING_SLASH
              + TWO_DIGITS
              + SLASH
              +FOUR_DIGITS)
NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                             NO_ONE_DAY
                             + OR
                             + NO_TWO_DAYS)

In [187]:
CENTURY = GROUP.format ('19' + OR + "20") + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
    DIGIT_SLASH,
    DIGIT_DASH,
    DIGIT_SPACE,
    LETTER_SPACE,
    COMMA_SPACE,
]))
YEAR_ONLY = NAMED.format("year_only",
                         YEAR_PREFIX + CENTURY)

In [188]:
IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join ([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)

In [189]:
DATE = NAMED.format ("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS, 
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))

In [190]:
def twentieth_century(date):
    """adds 19 to the year"""
    month, day, year = date.group(1).split(SLASH)
    year = "19{}".format(year)
    return SLASH.join([month, day,year])
    

In [191]:
def take_two(line):
    match = re.search(TWENTIETH_CENTURY, line)
    if match:
        return twentieth_century(match)
    return line


In [192]:
def extract_and_count(expression, data, name):
    """extract matches and says the count"""
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    print("'{}' matched {} rows".format(name,count))
    return extracted, count

In [193]:
numeric, numeric_count = extract_and_count(NUMERIC, data, 'numeric')
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, data, 'twentieth')
words, words_count = extract_and_count(WORDS, data, 'words')
backwards, backwards_count = extract_and_count(BACKWARDS, data, 'backwards')
contraction_data, contraction = extract_and_count(CONTRACTION, data, 'contraction')
no_day, no_day_count = extract_and_count(NO_DAY, data, 'no_day')
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, data, "no_day_numeric")
year_only, year_only_count = extract_and_count(YEAR_ONLY, data, "year_only")
leftovers, leftovers_count = extract_and_count(LEFTOVERS, data, "leftovers")

'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows


In [194]:
found = data.str.extractall(DATE)
total_found = len(found.date)
print("Total Found: {}".format(total_found))
print("Remaining: {}".format(len(data) - total_found))
print("Discrepancy: {}".format(total_found - (numeric_count
                                             + twentieth_count
                                             + words_count
                                             + backwards_count
                                             + contraction
                                             + no_day_count
                                             + no_day_numeric_count
                                             + year_only_count
                                             + leftovers_count)))

Total Found: 500
Remaining: 0
Discrepancy: 0


In [195]:
#total found: 500
#remaining: 0
#discrepancy: 0

In [196]:
missing = [label for label in data.index if label not in found.index.levels[0]]
try:
    print(missing[0], data.loc[missing[0]])
except IndexError:
    print("all rows matched")

all rows matched


In [197]:
#all rows matched

In [198]:
def clean(source, expression, replacement, sample = 5):
    print("Random Sample Before:")
    print(source.sample(sample))
    cleaned = source.str.replace(expression, replacement)
    print("\nRandom Sample After:")
    print(cleaned.sample(sample))
    print("\nCount of Cleaned: {}".format(len(cleaned)))
    assert len(source) == len(cleaned)
    return cleaned

In [199]:
def clean_punctuation(source, sample =5):
    print("Cleaning Punctuation")
    if any(source.str.contains(PUNCTUATION)):
        source = clean(source, PUNCTUATION, EMPTY_STRING)
    return source

In [200]:
LONG_TO_SHORT = dict(January = "Jan",
                    February = "Feb",
                    March = "Mar",
                    April = "Apr",
                    May = "May",
                    June = "Jun",
                    July = "Jul",
                    August = "Aug",
                    September = "Sep",
                    October = "Oct",
                    November = "Nov",
                    December = "Dec")
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                   + CLASS.format(LOWER_CASE)
                                   + ZERO_OR_MORE
                                   for month in LONG_TO_SHORT.values()])
def long_month_to_short(match):
    return match.group(match.lastindex)

In [201]:
def convert_long_months_to_short(source, sample = 5):
    return clean(source,
                LONG_TO_SHORT_EXPRESSION,
                long_month_to_short)

In [202]:
def add_month_date(match):
    return "01/01/" + match.group()

In [203]:
def add_january_one(source):
    return clean(source, YEAR_ONLY, add_month_date)

In [204]:
two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)
def two_digits(match):
    return "{:02}".format(int(match.group()))


In [205]:
def clean_two_digits (source, sample = 5):
    return clean(source, two_digit_expression, two_digits, sample)

In [206]:
def clean_two_digits_isolated(source, sample =5):
    return clean (source, ONE_OR_TWO_DIGITS, two_digits, sample)

In [207]:
digits = ("{:02}".format(month) for month in range(1,13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
def month_to_digits(match):
    return MONTH_TO_DIGITS[match.group()]


In [208]:
def convert_short_month_to_digits(source, sample = 5):
    return clean (source,
                 SHORT_MONTHS_EXRPRESSION,
                 month_to_digits,
                 sample)

In [209]:
def clean_months(source, sample=5):
    cleaned = clean_punctuation(source)
    print("Converting long months to short")
    cleaned = clean (cleaned, LONG_TO_SHORT_EXPRESSION,
                    long_month_to_short, sample)
    print("Converting short months to digits")
    cleaned = clean(cleaned, SHORT_MONTHS_EXPRESSION,
                   month_to_digits, sample)
    return cleaned

In [210]:
def frame_to_series(frame, index_source, samples=5):
    combined = frame.month + SLASH + frame.day + SLASH + frame.year
    combined.index = index_source.index
    print(combined.sample(samples))
    return combined

In [211]:
year_only_cleaned = add_january_one(year_only)

Random Sample Before:
     match
478  0        1993
481  0        1974
483  0        1995
486  0        1973
466  0        1981
Name: year_only, dtype: object

Random Sample After:
     match
481  0        01/01/1974
486  0        01/01/1973
472  0        01/01/2010
483  0        01/01/1995
470  0        01/01/1983
Name: year_only, dtype: object

Count of Cleaned: 15


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [212]:
leftovers_cleaned = add_january_one(leftovers)

Random Sample Before:
     match
488  0        1977
499  0        1980
457  0        2001
491  0        2009
463  0        2014
Name: leftovers, dtype: object

Random Sample After:
     match
465  0        01/01/1976
469  0        01/01/2003
484  0        01/01/2004
475  0        01/01/2015
485  0        01/01/1987
Name: leftovers, dtype: object

Count of Cleaned: 30


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [213]:
cleaned = pandas.concat([year_only_cleaned, leftovers_cleaned])
print(len(cleaned))

45


In [214]:
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)


Random Sample Before:
     match
447  0         7/1985
345  0        10/1973
406  0        12/1994
348  0        12/2005
414  0         4/2004
Name: no_day_numeric, dtype: object

Random Sample After:
     match
377  0        06/2001
372  0        05/1995
381  0        01/2014
360  0        12/2008
424  0        04/1979
Name: no_day_numeric, dtype: object

Count of Cleaned: 112


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [215]:
no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")

Random Sample Before:
     match
428  0        12/2007
450  0        01/1994
409  0        10/1994
373  0        08/1989
368  0        08/1986
Name: no_day_numeric, dtype: object

Random Sample After:
     match
412  0        03/01/1990
363  0        12/01/1975
357  0        10/01/1993
394  0        10/01/2001
442  0        09/01/1992
Name: no_day_numeric, dtype: object

Count of Cleaned: 112


In [216]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_numeric_cleaned])
assert len(cleaned) == no_day_numeric_count + original

In [217]:
print(len(cleaned))


157


In [218]:
no_day_cleaned = clean_months(no_day)


Cleaning Punctuation
Random Sample Before:
     match
313  0        Decemeber 1978
305  0              Feb 2000
315  0              Jun 1976
281  0              Aug 2004
228  0        September 1985
Name: no_day, dtype: object

Random Sample After:
     match
259  0            Nov 1979
300  0        January 1994
322  0        October 1991
304  0            Mar 2002
289  0            Sep 2007
Name: no_day, dtype: object

Count of Cleaned: 115
COnverting long months to short
Random Sample Before:
     match
263  0        September 1981
301  0              Dec 1992
265  0            April 1989
241  0              May 2004
230  0              May 1986
Name: no_day, dtype: object

Random Sample After:
     match
237  0        Feb 1976
269  0        Jul 1992
298  0        Jan 1993
257  0        Sep 2015
317  0        Mar 1975
Name: no_day, dtype: object

Count of Cleaned: 115
Converting short months to digits
Random Sample Before:
     match
332  0        Jun 1974
240  0        May 2011
264 

/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [219]:
no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")

Random Sample Before:
     match
252  0        01 2007
300  0        01 1994
310  0        10 1992
284  0        01 1987
271  0        08 2008
Name: no_day, dtype: object

Random Sample After:
     match
275  0        09/01/1984
315  0        06/01/1976
244  0        01/01/2013
237  0        02/01/1976
304  0        03/01/2002
Name: no_day, dtype: object

Count of Cleaned: 115


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [220]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_cleaned])
print(len(cleaned))


272


In [221]:
assert len(cleaned) == no_day_count + original


In [222]:
frame = pandas.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())
frame.head()

,day,month,year
0,24,Jan,2001
1,10,Sep,2004
2,26,May,1982
3,28,June,2002
4,06,May,1972


In [223]:
frame.day = clean_two_digits(frame.day)


Random Sample Before:
43    13
46    04
31    26
55    09
15    14
Name: day, dtype: object

Random Sample After:
4     06
44    21
42    01
36    19
8     28
Name: day, dtype: object

Count of Cleaned: 69


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [224]:
frame.month = clean_months(frame.month)


Cleaning Punctuation
COnverting long months to short
Random Sample Before:
31    May
28    Jan
33    Aug
42    Oct
39    Oct
Name: month, dtype: object

Random Sample After:
12    Feb
14    Oct
55    Dec
25    Mar
22    May
Name: month, dtype: object

Count of Cleaned: 69
Converting short months to digits
Random Sample Before:
62    Jan
1     Sep
28    Jan
50    Oct
63    Mar
Name: month, dtype: object

Random Sample After:
27    09
46    10
4     05
13    02
21    11
Name: month, dtype: object

Count of Cleaned: 69


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [225]:
backwards_cleaned = frame_to_series(frame, backwards)


     match
175  0        10/21/1983
141  0        05/30/2016
171  0        10/04/1972
128  0        06/28/2002
158  0        08/23/2000
dtype: object


In [226]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, backwards_cleaned])
assert len(cleaned) == original + backwards_count

In [227]:
print(len(cleaned))


341


In [228]:
frame = pandas.DataFrame(words.str.split().tolist(), columns="month day year".split())
print(frame.head())

      month  day  year
0     April  11,  1990
1       May  30,  2001
2       Feb  18,  1994
3  February  18,  1981
4  October.  11,  2013


In [229]:
frame.month = clean_months(frame.month)


Cleaning Punctuation
Random Sample Before:
6       July
27       Oct
28       May
13    August
20      Sep.
Name: month, dtype: object

Random Sample After:
27         Oct
16      August
24         May
26        June
7     December
Name: month, dtype: object

Count of Cleaned: 34
COnverting long months to short
Random Sample Before:
2           Feb
4       October
14    September
6          July
16       August
Name: month, dtype: object

Random Sample After:
18    Jul
20    Sep
7     Dec
22    Nov
0     Apr
Name: month, dtype: object

Count of Cleaned: 34
Converting short months to digits
Random Sample Before:
3     Feb
21    Aug
32    Jan
33    Sep
23    Jun
Name: month, dtype: object

Random Sample After:
33    09
27    10
24    05
30    07
32    01
Name: month, dtype: object

Count of Cleaned: 34


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [230]:
frame.day = clean_punctuation(frame.day)


Cleaning Punctuation
Random Sample Before:
19    11,
32    07,
16    11,
2     18,
15    25,
Name: day, dtype: object

Random Sample After:
32    07
5     24
22    11
30    25
3     18
Name: day, dtype: object

Count of Cleaned: 34


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [231]:
frame.head()


,month,day,year
0,04,11,1990
1,05,30,2001
2,02,18,1994
3,02,18,1981
4,10,11,2013


In [232]:
words_cleaned = frame_to_series(frame, words)


     match
210  0        08/11/1989
227  0        09/15/2011
209  0        07/25/1983
217  0        06/13/2011
200  0        07/26/1978
dtype: object


In [233]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, words_cleaned])
assert len(cleaned) == original + words_count
print(len(cleaned))

375


In [234]:
print(twentieth.iloc[21])
twentieth_cleaned = twentieth.str.replace(DASH, SLASH)
print(cleaned.iloc[21])

4-13-82
01/01/1991


In [235]:
frame = pandas.DataFrame(twentieth_cleaned.str.split(SLASH).tolist(),
                         columns=["month", "day", "year"])
print(frame.head())

  month day year
0    03  25   93
1     6  18   85
2     7   8   71
3     9  27   75
4     2   6   96


In [236]:
frame.month = clean_two_digits_isolated(frame.month)


Random Sample Before:
40     2
28    06
25    10
50    02
7     10
Name: month, dtype: object

Random Sample After:
62    12
54    07
36    08
81    09
63    04
Name: month, dtype: object

Count of Cleaned: 100


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [237]:
frame.day = clean_two_digits_isolated(frame.day)


Random Sample Before:
0     25
91    05
23    02
26    20
37    15
Name: day, dtype: object

Random Sample After:
98    04
7     24
38    12
0     25
63    22
Name: day, dtype: object

Count of Cleaned: 100


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [238]:
frame.head()

,month,day,year
0,03,25,93
1,06,18,85
2,07,08,71
3,09,27,75
4,02,06,96


In [239]:
frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())


Random Sample Before:
93    77
43    92
39    84
58    77
40    75
Name: year, dtype: object

Random Sample After:
58    1977
70    1982
86    1995
27    1987
34    1987
Name: year, dtype: object

Count of Cleaned: 100


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [240]:
twentieth_cleaned = frame_to_series(frame, twentieth)


     match
21   0        10/21/1979
79   0        10/04/1998
103  0        09/22/1982
76   0        12/05/1987
29   0        10/24/1986
dtype: object


In [241]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, twentieth_cleaned])


In [242]:
assert len(cleaned) == original + twentieth_count

In [243]:
print(numeric.head())

    match
14  0         5/24/1990
15  0         1/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
Name: numeric, dtype: object


In [244]:
has_dashes = numeric.str.contains(DASH)
print(numeric[has_dashes])

Series([], Name: numeric, dtype: object)


In [245]:
frame = pandas.DataFrame(numeric.str.split(SLASH).tolist(),
                         columns="month day year".split())
print(frame.head())


  month day  year
0     5  24  1990
1     1  25  2011
2    10  13  1976
3    07  25  1984
4    03  31  1985


In [246]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
15     7
14    01
11    08
4     03
2     10
Name: month, dtype: object

Random Sample After:
6     07
24    04
3     07
2     10
8     02
Name: month, dtype: object

Count of Cleaned: 25


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [247]:
frame.day = clean_two_digits_isolated(frame.day)


Random Sample Before:
24    27
13     3
20    11
0     24
16    14
Name: day, dtype: object

Random Sample After:
22    11
5     12
21    18
8     15
24    27
Name: day, dtype: object

Count of Cleaned: 25


/var/folders/vr/w4mk08vj2_l_l280wsytlh4w0000gn/T/ipykernel_22272/968746162.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned = source.str.replace(expression, replacement)


In [248]:
numeric_cleaned = frame_to_series(frame, numeric)


     match
64   0        07/29/1994
106  0        02/11/2006
15   0        01/25/2011
102  0        07/18/2002
94   0        12/08/1990
dtype: object


In [249]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, numeric_cleaned])
assert len(cleaned) == original + numeric_count
print(len(cleaned))

500


In [250]:
cleaned = pandas.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
])
print(len(cleaned))
print(cleaned.head())
assert len(cleaned) == len(data)

500
    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object


In [251]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


In [252]:
print(cleaned.head())
datetimes = pandas.to_datetime(cleaned, format="%m/%d/%Y")
print(datetimes.head())

    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object
    match
14  0       1990-05-24
15  0       2011-01-25
17  0       1976-10-13
24  0       1984-07-25
30  0       1985-03-31
dtype: datetime64[ns]


In [253]:
sorted_dates = datetimes.sort_values()
print(sorted_dates.head())

    match
9   0       1971-04-10
84  0       1971-05-18
2   0       1971-07-08
53  0       1971-07-11
28  0       1971-09-12
dtype: datetime64[ns]


In [257]:
print(sorted_dates.tail())

     match
231  0       2016-05-01
141  0       2016-05-30
186  0       2016-10-13
161  0       2016-10-19
413  0       2016-11-01
dtype: datetime64[ns]


In [ ]:
answer = pandas.Series(sorted_dates.index.labels[0])
print(answer.head())

In [ ]:
#0   9
#1   84
#2   2
#3   53
#4   28
#dtype: int16

In [ ]:
def date_sorter():
    return answer